In [275]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, max as max_
spark = SparkSession.builder.appName("Users By Average Session Time").getOrCreate()

In [276]:
df_input = spark.read.options(header='True', InferSchema='True').csv('facebook_web_log.csv')

In [277]:
##create 2 df having one action as page_load another as page_exist
## take the min and max based on user_id and action 

In [278]:
df_input.createOrReplaceTempView("fb")
df_page_exit= spark.sql("select user_id,timestamp,action,rn from "+ " (select user_id,timestamp,action, row_number() OVER (PARTITION BY user_id,timestamp ORDER BY timestamp ASC) as rn " +" FROM fb) tmp where action in('page_exit')")
df_page_load= spark.sql("select user_id,timestamp,action,rn from "+ " (select user_id,timestamp,action, row_number() OVER (PARTITION BY user_id,timestamp ORDER BY timestamp ASC) as rn " +" FROM fb) tmp where action in('page_load')")

In [279]:
df_page_load.show()
df_page_exit.show()

+-------+-------------------+---------+---+
|user_id|          timestamp|   action| rn|
+-------+-------------------+---------+---+
|      0|2019-04-25 13:30:15|page_load|  1|
|      0|2019-04-25 13:30:18|page_load|  1|
|      0|2019-04-28 14:30:10|page_load|  1|
|      0|2019-04-28 14:30:15|page_load|  1|
|      1|2019-04-25 13:40:00|page_load|  1|
|      1|2019-04-26 11:15:00|page_load|  1|
|      2|2019-04-25 13:41:21|page_load|  1|
|      3|2020-06-30 13:41:15|page_load|  1|
|      3|2020-06-30 14:20:12|page_load|  1|
+-------+-------------------+---------+---+

+-------+-------------------+---------+---+
|user_id|          timestamp|   action| rn|
+-------+-------------------+---------+---+
|      0|2019-04-25 13:31:40|page_exit|  1|
|      0|2019-04-28 15:31:40|page_exit|  1|
|      1|2019-04-25 13:40:35|page_exit|  1|
|      1|2019-04-26 11:15:35|page_exit|  1|
|      3|2020-06-30 15:20:11|page_exit|  1|
|      3|2020-06-30 16:44:11|page_exit|  1|
+-------+-------------------+--

In [280]:
df_page_load.createOrReplaceTempView("page_load")
df_load= spark.sql("select user_id,substring(timestamp,1,10) as _date,substring(timestamp,12,19) as _time,timestamp,action,rn from "+ " (select user_id,timestamp,action, row_number() OVER (PARTITION BY user_id,timestamp ORDER BY timestamp DESC) as rn " +" FROM page_load) tmp where rn = 1")
df_page_exit.createOrReplaceTempView("page_exit")
df_exit= spark.sql("select user_id,substring(timestamp,1,10) as _date,substring(timestamp,12,19) as _time,timestamp,action,rn from "+ " (select user_id,timestamp,action, row_number() OVER (PARTITION BY user_id,timestamp ORDER BY timestamp DESC) as rn " +" FROM page_exit) tmp where rn = 1")

In [281]:
df_load.show()
df_exit.show()

+-------+----------+--------+-------------------+---------+---+
|user_id|     _date|   _time|          timestamp|   action| rn|
+-------+----------+--------+-------------------+---------+---+
|      0|2019-04-25|13:30:15|2019-04-25 13:30:15|page_load|  1|
|      0|2019-04-25|13:30:18|2019-04-25 13:30:18|page_load|  1|
|      0|2019-04-28|14:30:10|2019-04-28 14:30:10|page_load|  1|
|      0|2019-04-28|14:30:15|2019-04-28 14:30:15|page_load|  1|
|      1|2019-04-25|13:40:00|2019-04-25 13:40:00|page_load|  1|
|      1|2019-04-26|11:15:00|2019-04-26 11:15:00|page_load|  1|
|      2|2019-04-25|13:41:21|2019-04-25 13:41:21|page_load|  1|
|      3|2020-06-30|13:41:15|2020-06-30 13:41:15|page_load|  1|
|      3|2020-06-30|14:20:12|2020-06-30 14:20:12|page_load|  1|
+-------+----------+--------+-------------------+---------+---+

+-------+----------+--------+-------------------+---------+---+
|user_id|     _date|   _time|          timestamp|   action| rn|
+-------+----------+--------+----------

In [282]:
df_load.createOrReplaceTempView("page_load_max")
df_page_load_max= spark.sql("select user_id as load_user_id,timestamp as page_load_timestamp,action,substring(timestamp,1,10) as page_load_date,substring(timestamp,12,19) as page_load_time,rn from "+ " (select user_id,timestamp,action, row_number() OVER (PARTITION BY user_id,_date ORDER BY _time DESC) as rn " +" FROM page_load_max) tmp where rn=1")
df_exit.createOrReplaceTempView("page_exit_max")
df_page_exit_max= spark.sql("select user_id as exit_user_id,timestamp as page_exit_timestamp,action,substring(timestamp,1,10) as page_exit_date,substring(timestamp,12,19) as page_exit_time,rn from "+ " (select user_id,timestamp,action, row_number() OVER (PARTITION BY user_id,_date ORDER BY _time ASC) as rn " +" FROM page_exit_max) tmp where rn=1")

In [283]:
df_page_load_max.show()
df_page_exit_max.show()

+------------+-------------------+---------+--------------+--------------+---+
|load_user_id|page_load_timestamp|   action|page_load_date|page_load_time| rn|
+------------+-------------------+---------+--------------+--------------+---+
|           0|2019-04-25 13:30:18|page_load|    2019-04-25|      13:30:18|  1|
|           0|2019-04-28 14:30:15|page_load|    2019-04-28|      14:30:15|  1|
|           1|2019-04-25 13:40:00|page_load|    2019-04-25|      13:40:00|  1|
|           1|2019-04-26 11:15:00|page_load|    2019-04-26|      11:15:00|  1|
|           2|2019-04-25 13:41:21|page_load|    2019-04-25|      13:41:21|  1|
|           3|2020-06-30 14:20:12|page_load|    2020-06-30|      14:20:12|  1|
+------------+-------------------+---------+--------------+--------------+---+

+------------+-------------------+---------+--------------+--------------+---+
|exit_user_id|page_exit_timestamp|   action|page_exit_date|page_exit_time| rn|
+------------+-------------------+---------+-------

In [284]:
df_page_output = df_page_load_max.join(df_page_exit_max, ((df_page_load_max.load_user_id == df_page_exit_max.exit_user_id) & (df_page_load_max.page_load_date == df_page_exit_max.page_exit_date)) ,'inner').selectExpr('load_user_id','page_load_timestamp','page_exit_timestamp','page_load_time','page_exit_time')

In [285]:
timeFmt = "yyyy-MM-dd' 'HH:mm:ss"
timeDiff = (F.unix_timestamp('page_exit_timestamp', format=timeFmt)- F.unix_timestamp('page_load_timestamp', format=timeFmt))
df_td = df_page_output.withColumn("DiffInSeconds", timeDiff)

In [286]:
df_output = df_td.groupBy('load_user_id').avg('DiffInSeconds').select(col("load_user_id").alias("user_id"),col("avg(DiffInSeconds)").alias("avg")).show()

+-------+------+
|user_id|   avg|
+-------+------+
|      1|  35.0|
|      3|3599.0|
|      0|1883.5|
+-------+------+

